# ETH Hackathon with Quantinuum: May 3 -- 5

Welcome to the ETH Hackathon!

In this task we will start from the basics and slowly build up a set of advanced tools and strategies for designing quantum circuits.
In the process, the goal is that we will also build up an intuition of what quantum computations is about, how quantum compilation works and where their computational
power is coming from!

### Summary
In this task you will learn about a circuit "normal form" and implement all the necessary
circuit transformations to convert a circuit into this normal form.
This normal form is of great interest for many reasons:
 - it gives a great intuition and mental model of what quantum computations are capable of: arbitrary circuits are decomposed into blocks, each with a very clear interpretation.
 - it is a quantum compiler enginneer's dream: on top of forming the basis of one of the most well-known optimisation strategies for error corrected circuits [1], very recent work has shown that this form is amenable reinforcement learning-based optimistion, with great results [2]! It also decomposes the compilation problem into a series of smaller problems that have yielded entire research avenues, from the optimisation of diagonal gates to Hadamard gate optimisation and the simplification of classical reversible circuits.
 - it uses a wide variety of gate types as well as slightly "advanced" quantum computation techniques such as classically controlled gates -- in summary, a great learning opportunity :D

### pyTKET

All code example and tasks will be expressed using [pytket](https://tket.quantinuum.com/api-docs/), a Python library for quantum circuit manipulation and compilation.
We will showcase some of its features and will be at your disposal for the entire duration of the hackathon in case of questions, feature requests, etc. Feedback is much appreciated, and there is a decent chance that whatever you request will be implemented in pytket (pure coincidence, of course)!

We recommend you use pytket for this task, but you are free to use whatever tool you like. If you feel more comfortable with qiskit or another framework feel free to use that and compare the results -- we cannot guarantee that you will be able to solve all the tasks as easily with it, however.

Are you new to `pytket` and quantum computing? We recommend you start with TODO: link this pytket beginner's guide.

### Notebook environment

We are providing notebook cloud environments for you to work in. The python environments will have `pytket`, `qiskit` and many other popular python libraries already installed and ready to use. You will even get free access to our remote quantum simulator, which you can use to run your circuits with accurate(-ish) noise simulation.

In [1]:
# Some basic utility functions that will make our code slightly easier to read.
# Have a look at it and make sure you understand what is going on.
#
# Use the TKET docs for reference -> https://tket.quantinuum.com/api-docs/
# In particular, you could start by looking at the pages
#  - All available gate types: https://tket.quantinuum.com/api-docs/optype.html
#  - Passes: https://tket.quantinuum.com/api-docs/passes.html


import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.circuit.display import get_circuit_renderer

import numpy as np

def print_circ(c: tk.Circuit):
    """ A simple utility to display circuits in notebooks """
    circuit_renderer = get_circuit_renderer()
    return circuit_renderer.render_circuit_jupyter(c)

def cliff_t_rebase() -> tkp.BasePass:
    """Pass to rebase single-qubit gates to the Clifford+T gateset.

    Concretely, single-qubit gates will be one of H, Z, S or T. Could also be
    updraded to handle X and V. Have a try!

    pyTKET won't do this for you automatically, because this is not a universal
    gateset (only approximately universal).

    For our purpose, if the decomposition is not exact, we raise an error
    """
    cx_replacement = tk.Circuit(2).CX(0, 1)
    def tk1_replacement(a, b, c, eps=1e-6):
        # make sure the phases are in the range [0, 4)
        a, b, c = a % 4, b % 4, c % 4
        ret = tk.Circuit(1)
        def add_phase(f: float):
            while f > eps:
                if f + eps > 1.:
                    ret.Z(0)
                    f -= 1
                elif f + eps > 0.5:
                    ret.S(0)
                    f -= 0.5
                elif f + eps > 0.25:
                    ret.T(0)
                    f -= 0.25
                else:
                    break
            return f
        rest_c = add_phase(c)
        ret.H(0)
        rest_b = add_phase(b)
        ret.H(0)
        rest_a = add_phase(a)

        if abs(rest_a) > eps or abs(rest_b) > eps or abs(rest_c) > eps:
            raise ValueError("Phases are not multiples of pi/4")
        return ret

    return tkp.RebaseCustom(
        {tk.OpType.CX, tk.OpType.H, tk.OpType.S, tk.OpType.T},
        cx_replacement=cx_replacement,
        tk1_replacement=tk1_replacement
    )



## Gatesets and their computational power

A first consideration when designing quantum circuits is the gateset in which the computation is expressed.
If we restrict ourselves to a very small subset of possible gate types, we will inevitably limit our expression power!

To give you an idea, consider the following two gate sets:
 - Only single-qubit gates: if all gates only apply to a single qubit, then no amount of quantum entanglement will be created: in physical terms, it is as if our atoms were space-like separated, decomposing the $2^n$-dimensional state space (for $n$ qubits) into a tensor product of $n$ 2-dimensional spaces. The result is a very small state space that can easily be simulated classically -- no quantum advantage here!
 - Only "classical reversible gates": choosing the gate set X (NOT gate) + CX (controlled-NOT gate) + CCX (controlled-controlled-NOT, or Toffoli, gate), we now obtain a set of operations that may span multiple qubits, and so entanglement will be created. However, as seen as unitary operations, all these gates are made of binary 0-1 matrix elements, and thus do not make use of the quantum property of "phase". The result is that these operations correspond exactly to classical arithmetic circuits! (you will see, we always use "classical" to refer to what normal computers can do).

In both cases above, we can introduce a "quantum resource", i.e. an additional gate type that suddenly increases the computational power of the circuits defined with it:
  - in the first case, we can add the CX gate, an entangling operation,
  - in the second case, we can add the H gate, a Hadamard gate, which is a single-qubit operation that splits a qubit in the computational `|0>` state into a superposition of two states `|0> + |1>`.
The result is "universal" quantum computation: i.e. it can be shown that those expanded gate sets are sufficient to express any unitary computation.

Note that in the two cases, what we considered the "quantum" resource was a matter of perspective: in one case, we considered the interaction of more than one qubit as the source of quantumness, whereas in the other case that same operation was viewed as a classical operation, that must be extended using "superposition" as a resource.
In each case, all computations _without_ the key quantum resource become classical, i.e. straightforward to simulate on normal CPUs.

## Clifford circuits
Another such split of computational power into "cheap and classical" + "quantum secret sauce" is the Clifford+T gate set.
This is particularly interesting in the context of using error correcting codes to protect quantum computations against noise: it turns out that
in popular codes, Cliffords can be performed cheaply whereas T-gates require dedicated state preparation factories on hardware, so called "magic states".

It is quite surprising at first that Clifford circuits are easy to simulate (a result known as the Gottesman-Knill theorem), as they include _both_
entanling and superposition primitives -- the confirmation that what is "quantumness" is subjective!
The gate set is the following:
 - the Hadamard gate ("superposition primitive")
 - the CX gate ("entanglement primitive")
 - the S gate ("pi/4 phase shift primitive")
In this case, the quantumness comes from introducing smaller phase shifts (you can view the Clifford circuits a discretised subset of possible quantum operations). It turns out that up to approximation (the gate set is "approximately universal"), it is sufficient to add the T gate to the mix, a pi/8 phase shift.

PyTKET comes with a couple of optimisation passes that can be used to simplify Clifford circuits, such as `CliffordPushThroughMeasures` and `CliffordSimp`.



In [2]:
# Consider the following circuit:
c = tk.Circuit(3).CX(0, 1).CX(1, 2).H(1).CX(1, 2)

# Apply the Clifford optimisation pass, followed by our custom rebase pass to express gates
# in our familiar gateset (TKET by default will use TK1 gates, its own parametric representation
# of arbitrary single-qubit gates). If you are into maths: it expresses any SU(2)
# operation as three axis rotations. We will just stick to non-parametrised gates.
cliff_opt = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])
cliff_opt.apply(c)

print_circ(c)


Note how TKET has reduced the number of CX gates (the ones in red, spanning two qubits), but has added a lot of "garbage" single qubit gates.
The reason for this is that we typically (at least on current hardware) care a lot
more about two-qubit gate count (CXs here) than single qubit gates.

**Task 1**. Play around with the circuit builder, the circuit representation and the [`CliffordSimp`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.CliffordSimp) pass. What is the smallest circuit you can find for which the number of CX gates gets decreased non-trivially by the pass?

Bonus: can you find a circuit for
which you can show that the circuit returned by `CliffordSimp` is not optimal in terms of CX gate count?

**Task 2**. Have a look at the [`CliffordPushThroughMeasures`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.CliffordPushThroughMeasures) pass. What is its purpose? Can you combine it with `CliffordSimp` to get even better results? Hint: use the [`c.measure_all()`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.measure_all) method -- what does it do?


TODO @Callum: what is a good way to execute circuits that include classical operations (the type you get from the CliffordPushThroughMeasure pass) at the end?

## Diagonal gates

Everything we have explored so far can be easily classically simulated -- boring!
Let us explore another interesting type of gate. This one is more "high level" meaning that you
won't find any hardware that implements anything close to it. Rather, it is a useful primitive
to use as we build more complex computation, that we will then ask pytket to optimise and express
as the type of "low-level" gates that we encountered above.

In [3]:
c = tk.Circuit(3)

# We build a 3qb gate with the following diagonal
# Note that the resulting diagonal matrix must be unitary, so all values must have amplitude 1.
diag = np.exp(1j * np.pi * np.arange(8) / 4)
diag_box = tkc.DiagonalBox(diag)
c.add_diagonal_box(diag_box, [0, 1, 2])

# This gate is a bit too complex too render nicely graphically, so the renderer just shows
# us a very helpful grey box... fair enough
print_circ(c)

Here is how we can ask TKET to decompose it to our previous gateset. It's magic!

In [4]:
box_decomp = tkp.SequencePass([tkp.DecomposeBoxes(), cliff_t_rebase()])

box_decomp.apply(c)
print_circ(c)


**Task 3.** Play around with the complex values on the diagonal unitary. What is the smallest number of CX gates you can obtain? What is the largest? Can you obtain more optimised results if you post-process the decomposition with some further passes such as `CliffordSimp` or another one?

Note that you will probably need to remove the `cliff_t_rebase` pass, as in some cases it won't find a valid decomposition. You could use `tkp.auto_rebase.auto_rebase_pass` to target a universal gateset of your choice, e.g. CX+Rz+Rx (i.e. CX and single-qubit rotations along the Z and X axes).

## Classical reversible circuits

The third type of gates we will look at are reversible classical circuits, which we have already shortly
mentioned in the introduction.
They are not universal for quantum computing: in fact, they can express exactly the same computations as boolean bit logic,
with the slight caveat that the operation must be reversible, i.e. invertible!
We thus cannot compute boolean AND (it only has one output for two inputs, so cannot be reversible),
but we can perform a NOT gate or the operation (x, y) -> (x, y + x mod 2) (the controlled-NOT aka CX gate).
The restriction to "reversible" operations comes directly from the physics: quantum computers manipulate atoms
following the laws of quantum mechanics, all of which are reversible -- the result is an overall computation
that must it too be reversible.

In practice, this reversibility restriction is not a problem: if necessary we can always add additional (qu)bits
to the input and output of gate operations such that i) the operation on the original set of (qu)bits is the desired (non-reversible) opertaion, while ii) at the same time being a reversible operation when viewed on a superset of (qu)bits.
And AND gate can for then be implemented by the operation (x, y, z) -> (x, y, z + x & y). If the input z is set to 0, then the output will be x & y. This gate is known as the controlled-controlled NOT, aka CCX.

In pyTKET, reversible classical operations are described using the `ToffoliBox`. Note that in the literature, the word "Toffoli" can confusingly be used to either refer to any reversible classical circuit or to refer specifically to the CCX gate.

In [5]:
def rev_or(a, b, c):
    """Add a | b to c"""
    return [a, b, ((a | b) +c) % 2]

print([([a, b, c], rev_or(a, b, c)) for a in (0, 1) for b in (0, 1) for c in (0, 1)])
tof_box = tkc.ToffoliBox([([a, b, c], rev_or(a, b, c)) for a in (0, 1) for b in (0, 1) for c in (0, 1)])

c = tk.Circuit(3)
c.add_toffolibox(tof_box, [0, 1, 2])
print_circ(c)

[([0, 0, 0], [0, 0, 0]), ([0, 0, 1], [0, 0, 1]), ([0, 1, 0], [0, 1, 1]), ([0, 1, 1], [0, 1, 0]), ([1, 0, 0], [1, 0, 1]), ([1, 0, 1], [1, 0, 0]), ([1, 1, 0], [1, 1, 1]), ([1, 1, 1], [1, 1, 0])]


**Task 4.** Write your own CCX using a reversible classical circuit. How does it get decomposed? Is it the same decomposition used for TKET's `OpType.CCX`?

## Classically controlled gates

Finally, we will look at one more circuit primitive before getting to the real task. This was just the warm up, really :P

This one isn't really a type of gate or circuit -- rather, it's an additional feature of most recent quantum computing hardware: we can measure a subset of the qubits to obtain a classical "bit" value, and then use that boolean
as a condition for some future quantum operation on other qubits that have not been measured yet.
These further operations will only be performed if the boolean value is set.
This feature is sometimes called classical feedforward, but we will use "classically controlled" here.

This turns out to be extremely useful! We can design circuits such that the measurement of a particular qubit informs us on which of two subspaces the global quantum state was projected into -- if the subspace that we find ourselves in is not the desired one, we can then choose to either abort the computation early, or better, perform some further quantum operations
that will correct the state to be in the appropriate subspace.

Two techniques that rely on these capabilities that you can read up on are "Measurement-based quantum computing" and "Repeat until success" circuits. TODO: Add references!

Here is how such a circuit can be constructed in pyTKET.

In [6]:
# Note that we now specify the number of qubits (as before), as well as the number of classical bits (here just 1)!
c = tk.Circuit(2, 1)
c.H(0).CX(0, 1)

# TODO @callum: can you fill in this example? I don't have internet right now so no docs :/

[H q[0]; CX q[0], q[1]; ]

We will see that we would like to classically control not just a single operation, but an entire sequence.
We could of course add the control bit to every single operation that requires it, but there is a simpler solution.

pyTKET allows you to box sequences of gates into subcircuits that you can then treat as a primitive gate operation within
your main circuit. This is called the `CircBox`.

In [7]:
c = tk.Circuit(2)

circ_box = tkc.CircBox(tk.Circuit(1).H(0).S(0))
c.add_circbox(circ_box, [0])
c.add_circbox(circ_box, [1])

## TODO @callum: can this be made classically controlled?

print_circ(c)

**Task 5.** Create your own CCX _again_, this time using controlled circboxes? How do you decompose it and how does it compare to your previous versions?

## Heyfron-Campbell Normal Form

We are finally ready to tackle your main task for this hackathon. Given any circuit as input, you must transform it into the following form (taken from [this paper](papers/1712.01557.pdf)).

![Screenshot 2024-04-30 at 10.49.56.png](imgs/heyfron-campbell.png)

The construction makes use of "ancilla" qubits, i.e. qubits that are used as resources during the computation. They are initialised at the beginning of the computation, used and then measured.
They are thus not part of the inputs and outputs of this computation.
The notation might be a bit unfamiliar, but every block in this circuit is something that we have just gone through:
 - "non-Clifford phase gate" is what we have called a Diagonal gate
 - "CNOT circuit" is a circuit made of CX gates only -- this is a special case of classical reversible circuits
 - The remaining "Clifford" blocks on the right are clifford circuits, classically controlled on a measurement of one of the ancilla qubits.

### Why is this form so cool?

First of all, it makes a very non-trivial statement: as cliffords and classical reversible circuits are easy, the only "classically hard" part in this form
is the diagonal gate on the computational states. Recall that this is a computation of the form
$$
\ket{b_1\cdots b_n} \mapsto e^{i\varphi_{b_1\cdots b_n}}\ket{b_1\cdots b_n},
$$
where $\ket{b_1\cdots b_n}$ is a computational basis state, and $\varphi_{b_1\cdots b_n}$ is a phase factor depending on the bits $b_1,\dots,b_n$.
In other words, it "just" inserts a phase on the computational basis states depending on the values of the ancilla qubits.
The CX circuit can be viewed as a sequence of sums of qubits modulo 2, i.e. of the type
$$
\ket{b_1\cdots b_n} \mapsto \ket{b_1\cdots b_{i-1} (b_i \oplus b_k) b_{i+1}\cdots b_n}
$$
for some $k \neq i$, something that we are familiar from boolean logic and arithmetic circuits.
Finally, the last Clifford blocks are a way to choose a "measurement basis".
In the simplest case, all ancilla qubits measure to 0, in which case the measurement is in the
computational basis.
In the other case, some classically simple clifford transformations are required before being
able to measure the state in the computational basis.

TODO: is the clifford part really easy? I am not sure how this would be simulated without state tomography!

This normal form also enables many different optimisation strategies.
In the regime where T gates are by far the most expensive, then only the first diagonal gate is
of interest: everything else is Clifford.

More subtly, this normal form also motivates the problem of finding circuits with low hadamard H gate counts: indeed the number of ancilla qubits required here is given by `h`, the number of such
gates. In a regime where the number of qubits is limited, this becomes a very relevant
optimisation cost function.

### Main task

This part is less guided than the previous parts, so feel free to read around, learn and ask us a lot
of questions! We recommend you start by reading the [original paper by Heyfron and Campbell](papers/1712.01557.pdf), and in particular section IIA and the figures 1 and 2.
Note that we are not going to do T-gate optimisation here (which is what the paper is mostly about),
but we will only do the initial circuit normalisation step.

Here is a rough guide on how to proceed:

#### "Hadamard Gadgetisation"
Build the circuit described in step 1 of figure 2 of the paper.
Using a [`CustomPass`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.CustomPass), you can then implement gadgetisation by replacing every H gate that appears in your circuit with the gadget circuit.
You may want to use [`n_gates_of_type`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.n_gates_of_type) to allocate the right number of ancilla qubits from the start and [`add_q_register`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_q_register) to maintain a distinction between your ancilla and non-ancilla qubits.

#### Pauli pushing
This is the most technical part. You will need to find out the circuit identities (commutation rules) 
that will allow you to commute Pauli gates from the middle of the circuit all the way to the end.
The following steps might help you:
1. Understand how an X gate can be commuted past any clifford gate.
2. Use the above commutation rules to find out what that implies for classically controlled operations.
3. Now you need to handle T gates that may appear in the circuit. Can you commute past them? What does your circuit end up looking like? Why is the resulting circuit Clifford?

TODO @callum: I actually haven't figured out the part with the non-clifford T gate!

#### Rince and repeat

Apply the transformation from the previous step to every classically controlled X gate in your circuit, starting from the end. The result will be an implementation of step 3 of figure 2.

### Correctness, simulation and benchmarking

Congratulations! You have almost made it to the end! At this point you should have a working implementation of the Heyfron-Campbell normal form. To conclude, we will make a simple correctness test and compare with other optimisation techniques.

The folder `circuits` in this repo contains a set of quantum circuits that are widely used to benchmark quantum circuit optimisation techniques, especially for T-gate count optimisation.
#### Correctness
Choose one of them and compare the simulation results using your Heyfron-Campbell normal form and the original circuit.
Do you get matching distributions/unitaries?

TODO @callum: how do we actually run such classically controlled circuits? Can we use a local simulator, or can we submit to Nexus?

#### Benchmarking
By combining your normal form with `DecomposeBoxes`, you indeed already get an optimisation pass that could be run on fault tolerant hardware (go find it)!
How do your results compare with the T gate count of the original circuit? 
What if you compare to the results on page 12 of the paper?

Finally, you can also compare CX gate count? Is your solution any good for this? You can for example compare to pyTKET's [`FullPeepholeOptimise`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.FullPeepholeOptimise) pass.